### 데이터 수집 = 한 페이지 크롤링

## 다나와 검색 페이지 접속

In [1]:
# selenium and chromedriver-autoinstaller 설치
! pip install selenium 
! pip install chromedriver-autoinstaller

In [34]:
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup

chromedriver_autoinstaller.install() # chromedriver 최신버전설치
driver = webdriver.Chrome()    # 브라우저 열기
url = "http://search.danawa.com/dsearch.php?query=무선청소기&tab=main"
driver.get(url)

In [35]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html , 'html.parser')

In [36]:
prod_items = soup.select('li.prod_item')
len(prod_items)

44

In [37]:
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

44

In [38]:
prod_items = soup.select('div.main_ad_prodlist > ul.product_list > li.prod_item')
len(prod_items)

0

In [39]:
prod_items = soup.select('div.main_prodlist.main_prodlist_list > ul.product_list > li.productItem')
len(prod_items)

0

In [40]:
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

44

In [41]:
spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')  # 수정버전(2022.05.19)
print(spec_list)

차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm


In [42]:
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip()
print(price)

46,010


In [43]:
prod_data = []
for prod_item in prod_items:
    if 'product-pot' in prod_item['class']:
        continue
    try:
        title = prod_item.select('p.prod_name > a')[0].text.strip()
        spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')
        price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip()

        prod_data.append([title, spec_list, price])
    except:
        pass
print(len(prod_data))
print(prod_data)

40
[['베이스어스 차량용 청소기 A3 (해외구매)', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm', '46,010'], ['LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm', '46,010'], ['LG전자 오브제컬렉션 코드제로 ThinQ A9S AT9201WC', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm', '46,010'], ['샤오미 드리미 V11 SE', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm', '46,010'], ['샤오미 LYDSTO 차량용 무선 청소기 H2 (해외구매)', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75d

In [44]:
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'product-pot' in prod_item['class']:
            continue
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
            spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')
            price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip()
            prod_data.append([title, spec_list, price])
        except:
            pass
    return prod_data

In [45]:
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

40


### 데이터 수집2 - 여러 페이지에 걸친 다나와 검색 페이지 크롤링

In [52]:
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=2945&defaultVaTab=417006&tab=goods'.format(keyword,page)
keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&volumeType=allvs&page=1&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=2945&defaultVaTab=417006&tab=goods


### 주피터 노트북의 진행표시줄 처리

In [47]:
! pip install tqdm

In [48]:
import time
from tqdm import tqdm_notebook
total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    time.sleep(5)

C:\Users\MINOS\AppData\Local\Temp\ipykernel_17756\983417027.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1, total_page + 1)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

keyword = '무선청소기'
total_page = 10
prod_data_total = []

for page in tqdm_notebook(range(1,total_page+1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    prod_data_total = prod_data_total + prod_item_list

C:\Users\MINOS\AppData\Local\Temp\ipykernel_17756\343483098.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1,total_page+1)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [61]:
! pip install pandas
! pip install openpyxl

In [62]:
import pandas as pd
import openpyxl
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('실습.xlsx', index = False)